In [1]:
# libraries importing
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import numpy as np
import os

from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

from process_data import process_data, get_single_df
from algorithms.Vanilla_LSTM import Vanilla_LSTM
from algorithms.Conv_AE import Conv_AE

## Data loading

In [2]:
datasets = process_data()

valve1_X =  datasets["valve1_X"]
valve1_y = datasets["valve1_y"]
valve2_X = datasets["valve2_X"]
valve2_y = datasets["valve2_y"]
other_anomaly_X = datasets["other_anomaly_X"]
other_anomaly_y = datasets["other_anomaly_y"]

In [3]:
combined_dataset_X, combined_dataset_y = get_single_df()

In [4]:
def test_train_split(df_X, df_y):
    size_train = int(df_X.shape[0]*0.8)
    size_test = df_X.shape[0] - size_train
    x_train = df_X[:size_train]
    y_train = df_y[:size_train].anomaly
    x_test = df_X[-size_test:]
    y_test = df_y[-size_test:].anomaly
    return x_train, y_train, x_test, y_test

In [5]:
# hyperparameters selection
N_STEPS = 120
Q = 0.65 # quantile for upper control limit (UCL) selection

# Generated training sequences for use in the model.
def create_sequences(values, time_steps=N_STEPS):
    output = []
    for i in range(len(values) - time_steps + 1):
        output.append(values[i : (i + time_steps)])
    return np.stack(output)

## LightGBM


In [6]:
model = lgb.LGBMClassifier()

### Combined dataset

In [7]:
x_train, y_train, x_test, y_test = test_train_split(combined_dataset_X, combined_dataset_y)
model.fit(x_train,y_train,eval_set=[(x_test,y_test),(x_train,y_train)],eval_metric=['rmse', 'l2', 'loggloss', 'mape'])

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 6593, number of negative: 21795
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000547 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1789
[LightGBM] [Info] Number of data points in the train set: 28388, number of used features: 8
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.232246 -> initscore=-1.195672
[LightGBM] [Info] Start training from score -1.195672


LGBMClassifier()

In [8]:
print('Training accuracy {:.4f}'.format(model.score(x_train,y_train)))
print('Testing accuracy {:.4f}'.format(model.score(x_test,y_test)))

Training accuracy 0.9523
Testing accuracy 0.7899


In [9]:
yhat = model.predict(x_test)

accuracy = accuracy_score(y_test, yhat)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat)
print('F1 score: %f' % f1)

Accuracy: 0.789911
Precision: 0.716019
Recall: 0.742783
F1 score: 0.729155


In [10]:
conf_matrix = metrics.confusion_matrix(y_test, yhat)

TN, FP, FN, TP = conf_matrix.ravel()

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate FAR false alarm rate
FPR = FP/(FP+TN)
# False negative rate MAR missing alarm rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)

print(f'False Alarm Rate: {FPR}')
print(f'Missing Alarm Rate: {FNR}')

False Alarm Rate: 0.18111490329920363
Missing Alarm Rate: 0.25721687638786084


In [11]:
print(metrics.classification_report(y_test,model.predict(x_test)))

              precision    recall  f1-score   support

         0.0       0.84      0.82      0.83      4395
         1.0       0.72      0.74      0.73      2702

    accuracy                           0.79      7097
   macro avg       0.78      0.78      0.78      7097
weighted avg       0.79      0.79      0.79      7097



### Valve 1 dataset

In [12]:
x_train, y_train, x_test, y_test = test_train_split(valve1_X, valve1_y)
model.fit(x_train,y_train,eval_set=[(x_test,y_test),(x_train,y_train)],eval_metric=['rmse', 'l2', 'loggloss', 'mape'])

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5074, number of negative: 9455
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000321 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1752
[LightGBM] [Info] Number of data points in the train set: 14529, number of used features: 8
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.349233 -> initscore=-0.622414
[LightGBM] [Info] Start training from score -0.622414


LGBMClassifier()

In [13]:
print('Training accuracy {:.4f}'.format(model.score(x_train,y_train)))
print('Testing accuracy {:.4f}'.format(model.score(x_test,y_test)))

Training accuracy 0.9791
Testing accuracy 0.9317


In [14]:
yhat = model.predict(x_test)

accuracy = accuracy_score(y_test, yhat)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat)
print('F1 score: %f' % f1)

Accuracy: 0.931737
Precision: 0.997982
Recall: 0.800810
F1 score: 0.888589


In [15]:
conf_matrix = metrics.confusion_matrix(y_test, yhat)

TN, FP, FN, TP = conf_matrix.ravel()

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate FAR false alarm rate
FPR = FP/(FP+TN)
# False negative rate MAR missing alarm rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)

print(f'False Alarm Rate: {FPR}')
print(f'Missing Alarm Rate: {FNR}')

False Alarm Rate: 0.0008340283569641367
Missing Alarm Rate: 0.19919028340080971


In [16]:
print(metrics.classification_report(y_test,model.predict(x_test)))

              precision    recall  f1-score   support

         0.0       0.91      1.00      0.95      2398
         1.0       1.00      0.80      0.89      1235

    accuracy                           0.93      3633
   macro avg       0.95      0.90      0.92      3633
weighted avg       0.94      0.93      0.93      3633



### valve 2 dataset

In [17]:
x_train, y_train, x_test, y_test = test_train_split(valve2_X, valve2_y)
model.fit(x_train,y_train,eval_set=[(x_test,y_test),(x_train,y_train)],eval_metric=['rmse', 'l2', 'loggloss', 'mape'])

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1122, number of negative: 2327
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000237 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1668
[LightGBM] [Info] Number of data points in the train set: 3449, number of used features: 8
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.325312 -> initscore=-0.729467
[LightGBM] [Info] Start training from score -0.729467


LGBMClassifier()

In [18]:
print('Training accuracy {:.4f}'.format(model.score(x_train,y_train)))
print('Testing accuracy {:.4f}'.format(model.score(x_test,y_test)))

Training accuracy 1.0000
Testing accuracy 0.8795


In [19]:
yhat = model.predict(x_test)

accuracy = accuracy_score(y_test, yhat)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat)
print('F1 score: %f' % f1)

Accuracy: 0.879490
Precision: 0.967846
Recall: 0.762025
F1 score: 0.852691


In [20]:
conf_matrix = metrics.confusion_matrix(y_test, yhat)

TN, FP, FN, TP = conf_matrix.ravel()

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate FAR false alarm rate
FPR = FP/(FP+TN)
# False negative rate MAR missing alarm rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)

print(f'False Alarm Rate: {FPR}')
print(f'Missing Alarm Rate: {FNR}')

False Alarm Rate: 0.021367521367521368
Missing Alarm Rate: 0.2379746835443038


In [21]:
print(metrics.classification_report(y_test,model.predict(x_test)))

              precision    recall  f1-score   support

         0.0       0.83      0.98      0.90       468
         1.0       0.97      0.76      0.85       395

    accuracy                           0.88       863
   macro avg       0.90      0.87      0.88       863
weighted avg       0.89      0.88      0.88       863



### Other anomalies dataset

In [22]:
x_train, y_train, x_test, y_test = test_train_split(other_anomaly_X, other_anomaly_y)
model.fit(x_train,y_train,eval_set=[(x_test,y_test),(x_train,y_train)],eval_metric=['rmse', 'l2', 'loggloss', 'mape'])

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4041, number of negative: 7947
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000295 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1767
[LightGBM] [Info] Number of data points in the train set: 11988, number of used features: 8
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.337087 -> initscore=-0.676302
[LightGBM] [Info] Start training from score -0.676302


LGBMClassifier()

In [23]:
print('Training accuracy {:.4f}'.format(model.score(x_train,y_train)))
print('Testing accuracy {:.4f}'.format(model.score(x_test,y_test)))

Training accuracy 0.9303
Testing accuracy 0.5415


In [24]:
yhat = model.predict(x_test)

accuracy = accuracy_score(y_test, yhat)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat)
print('F1 score: %f' % f1)

Accuracy: 0.541542
Precision: 0.000000
Recall: 0.000000
F1 score: 0.000000


/opt/homebrew/Caskroom/miniforge/base/envs/school_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
conf_matrix = metrics.confusion_matrix(y_test, yhat)

TN, FP, FN, TP = conf_matrix.ravel()

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate FAR false alarm rate
FPR = FP/(FP+TN)
# False negative rate MAR missing alarm rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)

print(f'False Alarm Rate: {FPR}')
print(f'Missing Alarm Rate: {FNR}')

False Alarm Rate: 0.0
Missing Alarm Rate: 1.0


/var/folders/zz/t2f5q7kj0bjdkxwz1v713q7w0000gn/T/ipykernel_5981/1278675977.py:10: RuntimeWarning: invalid value encountered in scalar divide
  PPV = TP/(TP+FP)
/var/folders/zz/t2f5q7kj0bjdkxwz1v713q7w0000gn/T/ipykernel_5981/1278675977.py:18: RuntimeWarning: invalid value encountered in scalar divide
  FDR = FP/(TP+FP)


In [26]:
print(metrics.classification_report(y_test,model.predict(x_test)))

              precision    recall  f1-score   support

         0.0       0.54      1.00      0.70      1623
         1.0       0.00      0.00      0.00      1374

    accuracy                           0.54      2997
   macro avg       0.27      0.50      0.35      2997
weighted avg       0.29      0.54      0.38      2997



/opt/homebrew/Caskroom/miniforge/base/envs/school_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/school_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/school_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this 

## Conv AE


In [27]:
model = Conv_AE()

### Combined dataset

In [28]:
x_train, y_train, x_test, y_test = test_train_split(combined_dataset_X, combined_dataset_y)
x_train_steps = create_sequences(np.array([row.values for i, row in x_train.iterrows()]), N_STEPS)
x_test_steps = create_sequences(np.array([row.values for i, row in x_test.iterrows()]), N_STEPS)

model.fit(x_train_steps)

# results predicting
residuals = pd.Series(np.sum(np.mean(np.abs(x_train_steps - model.predict(x_train_steps)), axis=1), axis=1))
UCL = residuals.quantile(Q)

884/884 [==============================] - 1s 1ms/step


In [29]:
# train prediction
cnn_residuals = pd.Series(np.sum(np.mean(np.abs(x_train_steps - model.predict(x_train_steps)), axis=1), axis=1))

# data i is an anomaly if samples [(i - timesteps + 1) to (i)] are anomalies
anomalous_data = cnn_residuals > UCL
anomalous_data_indices = []
for data_idx in range(N_STEPS - 1, len(x_train_steps) - N_STEPS + 1):
    if np.all(anomalous_data[data_idx - N_STEPS + 1 : data_idx]):
        anomalous_data_indices.append(data_idx)

yhat_train = pd.Series(data=0, index=x_train.index)
yhat_train.iloc[anomalous_data_indices] = 1


# test prediction
cnn_residuals = pd.Series(np.sum(np.mean(np.abs(x_test_steps - model.predict(x_test_steps)), axis=1), axis=1))

# data i is an anomaly if samples [(i - timesteps + 1) to (i)] are anomalies
anomalous_data = cnn_residuals > UCL
anomalous_data_indices = []
for data_idx in range(N_STEPS - 1, len(x_test_steps) - N_STEPS + 1):
    if np.all(anomalous_data[data_idx - N_STEPS + 1 : data_idx]):
        anomalous_data_indices.append(data_idx)

yhat_test = pd.Series(data=0, index=x_test.index)
yhat_test.iloc[anomalous_data_indices] = 1
  
  

219/219 [==============================] - 0s 1ms/step


In [30]:
print('Training accuracy {:.4f}'.format(accuracy_score(y_train, yhat_train)))
print('Testing accuracy {:.4f}'.format(accuracy_score(y_test, yhat_test)))

Training accuracy 0.6262
Testing accuracy 0.6193


In [31]:
accuracy = accuracy_score(y_test, yhat_test)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat_test)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat_test)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat_test)
print('F1 score: %f' % f1)

Accuracy: 0.619276
Precision: 0.000000
Recall: 0.000000
F1 score: 0.000000


/opt/homebrew/Caskroom/miniforge/base/envs/school_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
conf_matrix = metrics.confusion_matrix(y_test, yhat_test)

TN, FP, FN, TP = conf_matrix.ravel()

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate FAR false alarm rate
FPR = FP/(FP+TN)
# False negative rate MAR missing alarm rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)

print(f'False Alarm Rate: {FPR}')
print(f'Missing Alarm Rate: {FNR}')

False Alarm Rate: 0.0
Missing Alarm Rate: 1.0


/var/folders/zz/t2f5q7kj0bjdkxwz1v713q7w0000gn/T/ipykernel_5981/163082209.py:10: RuntimeWarning: invalid value encountered in scalar divide
  PPV = TP/(TP+FP)
/var/folders/zz/t2f5q7kj0bjdkxwz1v713q7w0000gn/T/ipykernel_5981/163082209.py:18: RuntimeWarning: invalid value encountered in scalar divide
  FDR = FP/(TP+FP)


In [33]:
print(metrics.classification_report(y_test,yhat_test))

              precision    recall  f1-score   support

         0.0       0.62      1.00      0.76      4395
         1.0       0.00      0.00      0.00      2702

    accuracy                           0.62      7097
   macro avg       0.31      0.50      0.38      7097
weighted avg       0.38      0.62      0.47      7097



/opt/homebrew/Caskroom/miniforge/base/envs/school_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/school_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/school_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this 

### Valve 1 dataset

In [34]:
x_train, y_train, x_test, y_test = test_train_split(valve1_X, valve1_y)
x_train_steps = create_sequences(np.array([row.values for i, row in x_train.iterrows()]), N_STEPS)
x_test_steps = create_sequences(np.array([row.values for i, row in x_test.iterrows()]), N_STEPS)

model.fit(x_train_steps)

# results predicting
residuals = pd.Series(np.sum(np.mean(np.abs(x_train_steps - model.predict(x_train_steps)), axis=1), axis=1))
UCL = residuals.quantile(Q)

451/451 [==============================] - 1s 1ms/step


In [35]:
# train prediction
cnn_residuals = pd.Series(np.sum(np.mean(np.abs(x_train_steps - model.predict(x_train_steps)), axis=1), axis=1))

# data i is an anomaly if samples [(i - timesteps + 1) to (i)] are anomalies
anomalous_data = cnn_residuals > UCL
anomalous_data_indices = []
for data_idx in range(N_STEPS - 1, len(x_train_steps) - N_STEPS + 1):
    if np.all(anomalous_data[data_idx - N_STEPS + 1 : data_idx]):
        anomalous_data_indices.append(data_idx)

yhat_train = pd.Series(data=0, index=x_train.index)
yhat_train.iloc[anomalous_data_indices] = 1


# test prediction
cnn_residuals = pd.Series(np.sum(np.mean(np.abs(x_test_steps - model.predict(x_test_steps)), axis=1), axis=1))

# data i is an anomaly if samples [(i - timesteps + 1) to (i)] are anomalies
anomalous_data = cnn_residuals > UCL
anomalous_data_indices = []
for data_idx in range(N_STEPS - 1, len(x_test_steps) - N_STEPS + 1):
    if np.all(anomalous_data[data_idx - N_STEPS + 1 : data_idx]):
        anomalous_data_indices.append(data_idx)

yhat_test = pd.Series(data=0, index=x_test.index)
yhat_test.iloc[anomalous_data_indices] = 1
  
  

110/110 [==============================] - 0s 1ms/step


In [36]:
print('Training accuracy {:.4f}'.format(accuracy_score(y_train, yhat_train)))
print('Testing accuracy {:.4f}'.format(accuracy_score(y_test, yhat_test)))

Training accuracy 0.5826
Testing accuracy 0.6601


In [37]:
accuracy = accuracy_score(y_test, yhat_test)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat_test)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat_test)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat_test)
print('F1 score: %f' % f1)

Accuracy: 0.660061
Precision: 0.000000
Recall: 0.000000
F1 score: 0.000000


/opt/homebrew/Caskroom/miniforge/base/envs/school_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
conf_matrix = metrics.confusion_matrix(y_test, yhat_test)

TN, FP, FN, TP = conf_matrix.ravel()

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate FAR false alarm rate
FPR = FP/(FP+TN)
# False negative rate MAR missing alarm rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)

print(f'False Alarm Rate: {FPR}')
print(f'Missing Alarm Rate: {FNR}')

False Alarm Rate: 0.0
Missing Alarm Rate: 1.0


/var/folders/zz/t2f5q7kj0bjdkxwz1v713q7w0000gn/T/ipykernel_5981/163082209.py:10: RuntimeWarning: invalid value encountered in scalar divide
  PPV = TP/(TP+FP)
/var/folders/zz/t2f5q7kj0bjdkxwz1v713q7w0000gn/T/ipykernel_5981/163082209.py:18: RuntimeWarning: invalid value encountered in scalar divide
  FDR = FP/(TP+FP)


In [39]:
print(metrics.classification_report(y_test,yhat_test))

              precision    recall  f1-score   support

         0.0       0.66      1.00      0.80      2398
         1.0       0.00      0.00      0.00      1235

    accuracy                           0.66      3633
   macro avg       0.33      0.50      0.40      3633
weighted avg       0.44      0.66      0.52      3633



/opt/homebrew/Caskroom/miniforge/base/envs/school_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/school_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/school_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this 

### Valve 2 dataset

In [40]:
x_train, y_train, x_test, y_test = test_train_split(valve2_X, valve2_y)
x_train_steps = create_sequences(np.array([row.values for i, row in x_train.iterrows()]), N_STEPS)
x_test_steps = create_sequences(np.array([row.values for i, row in x_test.iterrows()]), N_STEPS)

model.fit(x_train_steps)

# results predicting
residuals = pd.Series(np.sum(np.mean(np.abs(x_train_steps - model.predict(x_train_steps)), axis=1), axis=1))
UCL = residuals.quantile(Q)

105/105 [==============================] - 0s 1ms/step


In [41]:
# train prediction
cnn_residuals = pd.Series(np.sum(np.mean(np.abs(x_train_steps - model.predict(x_train_steps)), axis=1), axis=1))

# data i is an anomaly if samples [(i - timesteps + 1) to (i)] are anomalies
anomalous_data = cnn_residuals > UCL
anomalous_data_indices = []
for data_idx in range(N_STEPS - 1, len(x_train_steps) - N_STEPS + 1):
    if np.all(anomalous_data[data_idx - N_STEPS + 1 : data_idx]):
        anomalous_data_indices.append(data_idx)

yhat_train = pd.Series(data=0, index=x_train.index)
yhat_train.iloc[anomalous_data_indices] = 1


# test prediction
cnn_residuals = pd.Series(np.sum(np.mean(np.abs(x_test_steps - model.predict(x_test_steps)), axis=1), axis=1))

# data i is an anomaly if samples [(i - timesteps + 1) to (i)] are anomalies
anomalous_data = cnn_residuals > UCL
anomalous_data_indices = []
for data_idx in range(N_STEPS - 1, len(x_test_steps) - N_STEPS + 1):
    if np.all(anomalous_data[data_idx - N_STEPS + 1 : data_idx]):
        anomalous_data_indices.append(data_idx)

yhat_test = pd.Series(data=0, index=x_test.index)
yhat_test.iloc[anomalous_data_indices] = 1
  
  

24/24 [==============================] - 0s 2ms/step


In [42]:
print('Training accuracy {:.4f}'.format(accuracy_score(y_train, yhat_train)))
print('Testing accuracy {:.4f}'.format(accuracy_score(y_test, yhat_test)))

Training accuracy 0.7257
Testing accuracy 0.6987


In [43]:
accuracy = accuracy_score(y_test, yhat_test)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat_test)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat_test)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat_test)
print('F1 score: %f' % f1)

Accuracy: 0.698725
Precision: 1.000000
Recall: 0.341772
F1 score: 0.509434


In [44]:
conf_matrix = metrics.confusion_matrix(y_test, yhat_test)

TN, FP, FN, TP = conf_matrix.ravel()

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate FAR false alarm rate
FPR = FP/(FP+TN)
# False negative rate MAR missing alarm rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)

print(f'False Alarm Rate: {FPR}')
print(f'Missing Alarm Rate: {FNR}')

False Alarm Rate: 0.0
Missing Alarm Rate: 0.6582278481012658


In [45]:
print(metrics.classification_report(y_test,yhat_test))

              precision    recall  f1-score   support

         0.0       0.64      1.00      0.78       468
         1.0       1.00      0.34      0.51       395

    accuracy                           0.70       863
   macro avg       0.82      0.67      0.65       863
weighted avg       0.81      0.70      0.66       863



### Other anomalies dataset

In [46]:
x_train, y_train, x_test, y_test = test_train_split(other_anomaly_X, other_anomaly_y)
x_train_steps = create_sequences(np.array([row.values for i, row in x_train.iterrows()]), N_STEPS)
x_test_steps = create_sequences(np.array([row.values for i, row in x_test.iterrows()]), N_STEPS)

model.fit(x_train_steps)

# results predicting
residuals = pd.Series(np.sum(np.mean(np.abs(x_train_steps - model.predict(x_train_steps)), axis=1), axis=1))
UCL = residuals.quantile(Q)

371/371 [==============================] - 1s 1ms/step


In [47]:
# train prediction
cnn_residuals = pd.Series(np.sum(np.mean(np.abs(x_train_steps - model.predict(x_train_steps)), axis=1), axis=1))

# data i is an anomaly if samples [(i - timesteps + 1) to (i)] are anomalies
anomalous_data = cnn_residuals > UCL
anomalous_data_indices = []
for data_idx in range(N_STEPS - 1, len(x_train_steps) - N_STEPS + 1):
    if np.all(anomalous_data[data_idx - N_STEPS + 1 : data_idx]):
        anomalous_data_indices.append(data_idx)

yhat_train = pd.Series(data=0, index=x_train.index)
yhat_train.iloc[anomalous_data_indices] = 1


# test prediction
cnn_residuals = pd.Series(np.sum(np.mean(np.abs(x_test_steps - model.predict(x_test_steps)), axis=1), axis=1))

# data i is an anomaly if samples [(i - timesteps + 1) to (i)] are anomalies
anomalous_data = cnn_residuals > UCL
anomalous_data_indices = []
for data_idx in range(N_STEPS - 1, len(x_test_steps) - N_STEPS + 1):
    if np.all(anomalous_data[data_idx - N_STEPS + 1 : data_idx]):
        anomalous_data_indices.append(data_idx)

yhat_test = pd.Series(data=0, index=x_test.index)
yhat_test.iloc[anomalous_data_indices] = 1
  
  

90/90 [==============================] - 0s 1ms/step


In [48]:
print('Training accuracy {:.4f}'.format(accuracy_score(y_train, yhat_train)))
print('Testing accuracy {:.4f}'.format(accuracy_score(y_test, yhat_test)))

Training accuracy 0.6014
Testing accuracy 0.5415


In [49]:
accuracy = accuracy_score(y_test, yhat_test)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat_test)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat_test)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat_test)
print('F1 score: %f' % f1)

Accuracy: 0.541542
Precision: 0.000000
Recall: 0.000000
F1 score: 0.000000


/opt/homebrew/Caskroom/miniforge/base/envs/school_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [50]:
conf_matrix = metrics.confusion_matrix(y_test, yhat_test)

TN, FP, FN, TP = conf_matrix.ravel()

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate FAR false alarm rate
FPR = FP/(FP+TN)
# False negative rate MAR missing alarm rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)

print(f'False Alarm Rate: {FPR}')
print(f'Missing Alarm Rate: {FNR}')

False Alarm Rate: 0.0
Missing Alarm Rate: 1.0


/var/folders/zz/t2f5q7kj0bjdkxwz1v713q7w0000gn/T/ipykernel_5981/163082209.py:10: RuntimeWarning: invalid value encountered in scalar divide
  PPV = TP/(TP+FP)
/var/folders/zz/t2f5q7kj0bjdkxwz1v713q7w0000gn/T/ipykernel_5981/163082209.py:18: RuntimeWarning: invalid value encountered in scalar divide
  FDR = FP/(TP+FP)


In [51]:
print(metrics.classification_report(y_test,yhat_test))

              precision    recall  f1-score   support

         0.0       0.54      1.00      0.70      1623
         1.0       0.00      0.00      0.00      1374

    accuracy                           0.54      2997
   macro avg       0.27      0.50      0.35      2997
weighted avg       0.29      0.54      0.38      2997



/opt/homebrew/Caskroom/miniforge/base/envs/school_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/school_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/school_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this 